# The Preprocessing Pipeline ⚙️
This notebook demonstrates the implementation of an innovative preprocessing pipeline for native tree images captured using a DJI Mavic 3M drone.

### Basic Imports

In [1]:
import subprocess
from PIL import Image
import numpy as np
import cv2
import glob
import os
from scipy.ndimage import gaussian_filter
import math
from itertools import cycle , islice

## First steps of the image processing pipeline 

This first part of the script defines a set of functions that work together to correct and align images based on camera calibration metadata.
Here's a breakdown of how each function contributes to the full workflow:

1. **`get_xml_metadata(imgPath)`**  
   Extracts metadata from the image file using ExifTool. This metadata includes calibration parameters such as optical center, vignetting coefficients, distortion data, and homography matrix.

2. **`vig_correct(image_path, infoDict)`**  
   Applies vignette correction to the image using a polynomial model. The correction is based on distance from the optical center and the vignetting coefficients retrieved from the metadata.

3. **`undistort(new_img, infoDict)`**  
   Corrects geometric distortion in the image using the camera matrix and distortion coefficients provided in the metadata.

4. **`align_phase_rotation(new_img, infoDict)`**  
   Applies a homographic transformation to align the image's perspective, correcting for phase and rotational differences due to camera setup.

5. **`crop_center(img, crop_tamanho)`**  
   Crops a centered square region of the image with the given size. This focuses the output on the most relevant region of the image.

6. **`zoom_center(img, zoom_factor=1.5)`**  
   For `.JPG` images (which have a different resolution), it crops and zooms the center area to match the target reference size.

7. **`process_image(imgPath)`**  
   The main pipeline that orchestrates all steps:
   - Extracts metadata.
   - Handles `.JPG` images with a separate zooming and resizing flow.
   - Applies vignette correction, distortion correction, and alignment for other formats.
   - Crops the central region to finalize the processed image.

In [2]:
def get_xml_metadata(imgPath):
  """
      get XML metadata
      This function retrieves metadata from an image file using ExifTool.
      Parameters:
      - imgPath: Path to the image file.
      Returns:
      - infoDict: Dictionary containing metadata tags and their values.
  """

  infoDict = {}
  exifToolPath = 'exiftool'
  ''' use Exif tool to get the metadata '''
  process = subprocess.Popen([exifToolPath,imgPath],stdout=subprocess.PIPE, stderr=subprocess.STDOUT,universal_newlines=True)
  ''' get the tags in dict '''
  for tag in process.stdout:
      line = tag.strip().split(':')
      infoDict[line[0].strip()] = line[-1].strip()
  return infoDict


def vig_correct(image_path, infoDict):
  """
    Vignette Correction
    This function applies a vignette correction to an image based on the provided calibration data.
    Parameters:
    - image_path: Path to the image file.
    - infoDict: Dictionary containing calibration data, including:
        - 'Calibrated Optical Center X': X coordinate of the optical center.
        - 'Calibrated Optical Center Y': Y coordinate of the optical center.
        - 'Vignetting Data': Vignetting coefficients.
    Returns:
    - corrected_img: The vignette-corrected image as a NumPy array.
  """

  centerX = int(float(infoDict['Calibrated Optical Center X']))
  centerY = int(float(infoDict['Calibrated Optical Center Y']))
  k = [float(elem.strip(",")) for elem in infoDict['Vignetting Data'].split()]

  # Load the image
  image = Image.open(image_path)
  np_img = np.array(image, dtype=np.uint16)
  rows, cols = np_img.shape[:2]

  # Create meshgrid of coordinates
  y_coords, x_coords = np.meshgrid(np.arange(rows), np.arange(cols), indexing='ij')

  # Calculate the distance r for each pixel
  r = np.sqrt((x_coords - centerX)**2 + (y_coords - centerY)**2)

  # Calculate the correction factor
  correction_factor = (
      k[5] * r**6 +
      k[4] * r**5 +
      k[3] * r**4 +
      k[2] * r**3 +
      k[1] * r**2 +
      k[0] * r +
      1.0
  )

  # Apply the correction factor to the image
  corrected_img = np_img * correction_factor[..., np.newaxis] if np_img.ndim == 3 else np_img * correction_factor
  return corrected_img.astype('uint16')


def undistort(new_img, infoDict):
  """
    Distortion correction
    This function undistorts an image using camera calibration data.
    Parameters:
    - new_img: The input image to be undistorted.
    - infoDict: Dictionary containing calibration data, including:
        - 'Calibrated Optical Center X': X coordinate of the optical center.
        - 'Calibrated Optical Center Y': Y coordinate of the optical center.
        - 'Dewarp Data': Distortion coefficients.
    Returns:
    - dst: The undistorted image as a NumPy array.
  """

  centerX = int(float(infoDict['Calibrated Optical Center X']))
  centerY = int(float(infoDict['Calibrated Optical Center Y']))

  dewarp_args = [float(elem) for elem in infoDict['Dewarp Data'].split(";")[1].split(",")]
  dist_coeffs = np.asarray(dewarp_args[4:])
  tuple0 = (dewarp_args[0], 0, centerX + dewarp_args[2])
  tuple1 = (0, dewarp_args[1], centerY + dewarp_args[3])
  tuple2 = (0, 0, 1)
  camera_matrix = np.asarray([tuple0, tuple1, tuple2])

  h,  w = new_img.shape[:2]
  _, roi=cv2.getOptimalNewCameraMatrix(camera_matrix,dist_coeffs,(w,h),1,(w,h))

  dst = cv2.undistort(new_img, camera_matrix, dist_coeffs, None, camera_matrix)
  # crop the image
  x,y,w,h = roi
  dst = dst[y:y+h, x:x+w]
  return dst


def align_phase_rotation(new_img, infoDict):
  """
    Alignment of the phase and rotation differences caused by different camera locations and
    optical accuracy

    This function aligns the phase and rotation differences in an image based on calibration data.
    Parameters:
    - new_img: The input image to be aligned.
    - infoDict: Dictionary containing calibration data, including:
        - 'Calibrated Optical Center X': X coordinate of the optical center.
        - 'Calibrated Optical Center Y': Y coordinate of the optical center.
        - 'Calibrated H Matrix': Homography matrix for perspective transformation.
    Returns:
    - new_img: The aligned (to an ideal plane) image as a NumPy array.
  """
  
  rows, cols = new_img.shape
  H = np.asarray([float(elem) for elem in infoDict['Calibrated H Matrix'].split(",")]).reshape(3,3)
  new_img = cv2.warpPerspective(new_img, H, (cols, rows))
  return new_img


def crop_center(img, crop_tamanho):
    """
        Crop the center of the image.
        Parameters:
            - img: The input image to be cropped.
            - crop_tamanho: The size of the square crop.
        Returns:
            - imagem_cropped: The cropped image as a NumPy array.
    """

    # get the dimensions of the image
    altura = img.shape[0]
    largura = img.shape[1]

    # calculate the starting and ending coordinates for the crop
    start_x = largura // 2 - crop_tamanho // 2
    start_y = altura // 2 - crop_tamanho // 2
    end_x = start_x + crop_tamanho
    end_y = start_y + crop_tamanho

    # crop the image
    imagem_cropped = img[start_y:end_y, start_x:end_x]
    return imagem_cropped


def zoom_center(img, zoom_factor=1.5):
    """
        Zoom in on the center of the image.
        Parameters:
            - img: The input image to be zoomed.
            - zoom_factor: The factor by which to zoom in.
        Returns:
            - img_zoomed: The zoomed image as a NumPy array.
    """

    y_size = img.shape[0]
    x_size = img.shape[1]

    # define new boundaries
    x1 = int(0.5*x_size*(1-1/zoom_factor))
    x2 = int(x_size-0.5*x_size*(1-1/zoom_factor))
    y1 = int(0.5*y_size*(1-1/zoom_factor))
    y2 = int(y_size-0.5*y_size*(1-1/zoom_factor))

    # first crop image then scale
    img_cropped = img[y1:y2,x1:x2]
    return cv2.resize(img_cropped, None, fx=zoom_factor, fy=zoom_factor)


def process_image(imgPath):
    """
        Process the image by applying vignette correction, undistortion, and alignment.
        Parameters:
            - imgPath: Path to the image file.
        Returns:
            - new_img: The processed image as a NumPy array.
    """
    
    # constant for the reference image size
    IMG_REF_SHAPE = (2570, 1925)

    # get xml metadata for camera corrections
    infoDict = get_xml_metadata(imgPath)

    # custom pipeline for jpg images, because they have a different resolution 
    if imgPath[-3:] == 'JPG':
            new_img = cv2.imread(imgPath)
            new_img = zoom_center(new_img, 1.3)
            new_img = cv2.resize(new_img, IMG_REF_SHAPE)
            new_img = crop_center(new_img, 1500)
            return new_img


    # apply vignette correction
    new_img = vig_correct(imgPath, infoDict)

    # undistort image
    new_img = undistort(new_img, infoDict)

    # align phase and rotation
    new_img = align_phase_rotation(new_img, infoDict)
    
    # crop center
    new_img = crop_center(new_img, 1500)

    return new_img

### Second part: Image Smoothing, Edge Detection, and Alignment using ECC

Now that the images are properly calibrated and show roughly the same area, we need to align the images based on structural features. Here's how each function contributes to the process:

8. **`smooth_image`**: Applies a Gaussian filter to reduce image noise and detail. This helps enhance the performance of edge detection by minimizing irrelevant high-frequency content.

9. **`edge_detection`**: Uses the Sobel operator to compute gradients along the x and y directions and calculates the gradient magnitude. The result highlights the edges in the image, which are crucial features for alignment.

10. **`align_images_using_ecc`**: Aligns a `target_image` to a `reference_image` using the Enhanced Correlation Coefficient (ECC) algorithm. The function:
  - Converts JPG images to grayscale (if applicable).
  - Applies smoothing and edge detection to both images.
  - Defines a homography warp model and calculates a transformation matrix using `cv2.findTransformECC`.
  - Applies the computed warp matrix to the `target_image`, aligning it with the `reference_image`.

This pipeline is particularly useful when working with multi-temporal or multi-spectral images where direct pixel-wise comparison may not be sufficient due to varying intensity distributions or minor misalignments.

In [3]:
def smooth_image(image, sigma=1):
    """
        Apply Gaussian smoothing to the image.
        Parameters:
            - image: The input image to be smoothed.
            - sigma: Standard deviation for Gaussian kernel.
        Returns:
            - smoothed_image: The smoothed image as a NumPy array.
    """
    return gaussian_filter(image, sigma=sigma)


def edge_detection(image):
    """
        Apply Sobel filter to detect edges in the image.
        Parameters:
            - image: The input image to be processed.
        Returns:
            - magnitude: The magnitude of the gradient, representing edge strength.
    """

    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = cv2.magnitude(grad_x, grad_y)
    return magnitude


def align_images_using_ecc(reference_image, target_image, jpg=False):
    """
        Align two images using the Enhanced Correlation Coefficient (ECC) algorithm.
        Parameters:
            - reference_image: The reference image to align to.
            - target_image: The target image to be aligned.
            - jpg: Boolean indicating if the target image is a JPG file.
        Returns:
            - aligned_image: The aligned target image as a NumPy array.
    """
    # JPG images do not have the same exif data as the reference TIFF images,
    # therefore they need a different treatment
    if jpg:
        # save colored jpg
        colored_jpg = target_image
        # create temporary greyscale img
        target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY) 
    
    # smooth the images
    reference_image_smoothed = smooth_image(reference_image)
    target_image_smoothed = smooth_image(target_image)

    # apply edge detection
    base_edges = edge_detection(reference_image_smoothed)
    target_edges = edge_detection(target_image_smoothed)

    # convert images to float32 for ECC algorithm
    base_edges = base_edges.astype(np.float32)
    target_edges = target_edges.astype(np.float32)
    
    # define the motion model
    warp_mode = cv2.MOTION_HOMOGRAPHY
    warp_matrix = np.eye(3, 3, dtype=np.float32)

    # set the number of iterations and termination criteria
    if jpg:
        # increase iterations for jpg images, beacause they went through lees processing
        number_of_iterations = 500 
    else:
        number_of_iterations = 25
    termination_eps = 1e-10

    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations, termination_eps)


    # apply the ECC algorithm to find the warp matrix
    cc, warp_matrix = cv2.findTransformECC(base_edges, target_edges, warp_matrix, warp_mode, criteria)
    
    if jpg:
        target_image = colored_jpg

    # warp the target image to align with the base image
    aligned_image = cv2.warpPerspective(target_image, warp_matrix, (reference_image.shape[1], reference_image.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    
    return aligned_image

### Putting It All Together: Image Preprocessing Pipeline Driver

This section orchestrates the entire image preprocessing pipeline, combining image selection, processing, alignment, normalization, cropping, and saving into a single loop. Here's a breakdown of the workflow:

- **Directory Setup & File Collection**:  
  The script changes the working directory to `./raw-imgs` and collects all `.JPG` and `.TIF` files using `glob`. The file list is sorted to maintain consistent band order.

- **Batch Processing with Sliding Window**:  
  Using a sliding window (`slc = 5`), the script iterates through the images in sets of five — assumed to be in the order `[JPG, G, NIR, R, RE]`.

- **Reference Selection & Alignment**:  
  The **G band** image is chosen as the reference (`ref_img`), and the remaining bands are aligned to it using the ECC-based alignment function:
  - `target_jpg` is aligned with the `jpg=True` flag (special grayscale treatment).
  - `target_nir`, `target_r`, and `target_re` are aligned normally.

- **Normalization and Cropping**:  
  Each aligned image is normalized to the 8-bit range `[0, 255]` and then cropped to a 1000x1000 center square for consistency and comparability.

- **Saving Preprocessed Images**:  
  All processed and aligned images are saved into the `preprocessed-imgs` folder with a `processed-` prefix.

- **Progress Logging**:  
  After each batch of 5 images is processed and saved, a status message is printed to the console.

This driver code ties together all prior functions and forms the backbone of the preprocessing routine, ensuring the images are ready for downstream analysis (e.g., model training or feature extraction).


In [4]:
# DRIVER
IMG_REF_SHAPE = (2570, 1925)

# Take relative paths, because chenged os current dir
os.chdir("./raw-imgs")
types = ('*.JPG', '*.TIF') # the tuple of file types
files_grabbed = []
for files in types:
    files_grabbed.extend(glob.glob(files))
    
input_imgs = sorted(files_grabbed)

i = cycle(input_imgs)
slc = 5
for _ in range(math.ceil(len(input_imgs)/slc)):
    cur_imgs = list(islice(i,slc))
    
    ref_img = process_image(cur_imgs[1]) # G BAND -> REFERENCE
    target_jpg = process_image(cur_imgs[0])
    target_nir = process_image(cur_imgs[2])
    target_r = process_image(cur_imgs[3])
    target_re = process_image(cur_imgs[4])

    # align image to G BAND
    aligned_jpg_image = align_images_using_ecc(ref_img, target_jpg, True)
    aligned_nir_image = align_images_using_ecc(ref_img, target_nir)
    aligned_r_image = align_images_using_ecc(ref_img, target_r)
    aligned_re_image = align_images_using_ecc(ref_img, target_re)

    # remember to crop and normalize reference/target img here:
    # normalize the image to range 0 to 255 and convert to uint8
    aligned_jpg_image = cv2.normalize(aligned_jpg_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    aligned_nir_image = cv2.normalize(aligned_nir_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    aligned_r_image = cv2.normalize(aligned_r_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    aligned_re_image = cv2.normalize(aligned_re_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    aligned_g_image = cv2.normalize(ref_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # crop again
    aligned_jpg_image = crop_center(aligned_jpg_image, 1000)
    aligned_nir_image = crop_center(aligned_nir_image, 1000)
    aligned_r_image = crop_center(aligned_r_image, 1000)
    aligned_re_image = crop_center(aligned_re_image, 1000)
    aligned_g_image = crop_center(aligned_g_image, 1000)

    # save
    cv2.imwrite(f"{os.path.dirname(os.getcwd())}/preprocessed-imgs/processed-{cur_imgs[0]}", aligned_jpg_image)
    cv2.imwrite(f"{os.path.dirname(os.getcwd())}/preprocessed-imgs/processed-{cur_imgs[1]}", aligned_g_image)
    cv2.imwrite(f"{os.path.dirname(os.getcwd())}/preprocessed-imgs/processed-{cur_imgs[2]}", aligned_nir_image)
    cv2.imwrite(f"{os.path.dirname(os.getcwd())}/preprocessed-imgs/processed-{cur_imgs[3]}", aligned_r_image)
    cv2.imwrite(f"{os.path.dirname(os.getcwd())}/preprocessed-imgs/processed-{cur_imgs[4]}", aligned_re_image)

    
    print(f"images {cur_imgs[0]} - {cur_imgs[4]} processed and saved")


images DJI_20250127100947_0154_D.JPG - DJI_20250127100947_0154_MS_RE.TIF processed and saved
images DJI_20250127100949_0155_D.JPG - DJI_20250127100949_0155_MS_RE.TIF processed and saved
images DJI_20250127100952_0156_D.JPG - DJI_20250127100952_0156_MS_RE.TIF processed and saved
images DJI_20250127100954_0157_D.JPG - DJI_20250127100954_0157_MS_RE.TIF processed and saved
images DJI_20250127100957_0158_D.JPG - DJI_20250127100957_0158_MS_RE.TIF processed and saved
images DJI_20250127101000_0159_D.JPG - DJI_20250127101000_0159_MS_RE.TIF processed and saved
images DJI_20250127101002_0160_D.JPG - DJI_20250127101002_0160_MS_RE.TIF processed and saved
images DJI_20250127101005_0161_D.JPG - DJI_20250127101005_0161_MS_RE.TIF processed and saved
images DJI_20250127101007_0162_D.JPG - DJI_20250127101007_0162_MS_RE.TIF processed and saved
images DJI_20250127101010_0163_D.JPG - DJI_20250127101010_0163_MS_RE.TIF processed and saved
images DJI_20250127101012_0164_D.JPG - DJI_20250127101012_0164_MS_RE.T